## Import packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import cline_analysis as ca
import pandas as pd
import seaborn as sns
import datetime
import os
from scipy.signal import medfilt
import functools
from scipy.optimize import bisect
from scipy import stats

sns.set_style("whitegrid")
sns.set_style("ticks")
%matplotlib qt
%config InlineBackend.figure_format = 'svg'
plt.matplotlib.rcParams['svg.fonttype'] = 'svgfont' # fonts will be recognized by Adobe Illustrator

## Load data

In [4]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Jutai/csv_files/'
fnames,clxs,clys,rbxs,lbxs,rbys,lbys,curvatures,ages,widths,dates = ca.load_data(dirname)

In [5]:
fnames

['Jutai_19890805.csv', 'Jutai_20170802.csv']

In [6]:
dates

[datetime.datetime(1989, 8, 5, 0, 0), datetime.datetime(2017, 8, 2, 0, 0)]

## Get migration rate

In [7]:
ts1 = 0 # first timestep
ts2 = 1 # second timestep

d = dates[ts2]-dates[ts1]
years = d.days/365.0

x = np.array(clxs[ts1])
y = np.array(clys[ts1])

xn = np.array(clxs[ts2])
yn = np.array(clys[ts2])

migr_rate, migr_sign, p, q = ca.get_migr_rate(x,y,xn,yn,years,0)

In [110]:
migr_rate = medfilt(savgol_filter(migr_rate,41,3),kernel_size=5) # smoothing
curv,s = ca.compute_curvature(x,y)
curv = medfilt(savgol_filter(curv,71,3),kernel_size=5) # smoothing

In [111]:
# set intervals affected by cu=toffs to NaN - specific to Jutai river 
migr_rate[1086:1293] = np.NaN

In [112]:
plt.figure()
plt.plot(migr_rate)

## Read 'valid' inflection points and corresponding points of zero migration from CSV file

In [113]:
df = pd.read_csv('Jutai_LT05_L1TP_003063_19890805_20170202_01_T1_inflection_and_zero_migration_indices.csv')
LZC = np.array(df['index of inflection point'])
LZM = np.array(df['index of zero migration'])

In [114]:
# indices of bends affected by low erodibility and cutoffs (these have been picked manually)
erodibility_inds = [69,115,117,119,163,189,191,204,218]
cutoff_inds = [7,8,9,14,15,29,30,50,51,58,59,185,194,209,210]

## Plot curvature and migration rate series side-by-side

In [115]:
# plot curvature and migration rate along the channel

W = np.nanmean(widths[0]) # mean channel width

fig, ax1 = plt.subplots(figsize=(25,4))
plt.tight_layout()

curv_scale = 0.6
migr_scale = 3
y1 = curv_scale
y2 = -3*curv_scale
y3 = 3*migr_scale
y4 = -migr_scale

y5 = -2*curv_scale
y6 = 2*migr_scale

for i in range(0,len(LZC)-1,2):
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,0.85,0.85],edgecolor='k',zorder=0)

deltas = 25.0
ax1.fill_between(s, 0, curv*W)
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_rate, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y2,y1)
ax2.set_ylim(y4,y3)
ax1.set_xlim(0,s[-1])

for i in erodibility_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[1.0,0.85,0.85],edgecolor='k',zorder=0) 
    
for i in cutoff_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,1.0,0.85],edgecolor='k',zorder=0) 
    
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y1,y1,0,y5,y2,y2,y5,0]
        ax1.fill(xcoords,ycoords,color='w') 
        
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y3,y3,y6,0,y4,y4,0,y6]
        ax2.fill(xcoords,ycoords,color='w') 

for i in range(0,len(LZC)-1,2):
    ax1.text(s[LZC[i]],0.5,str(i),fontsize=12)

## Estimate lag between curvature and migration rate

In [116]:
# plot widths and boundary between the two segments
plt.figure()
plt.plot(s,widths[0])
plt.plot([s[9846],s[9846]],[0,800],'r')

In [117]:
# first segment
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(widths[0][:9846])

123.26877138353407

In [118]:
# second segment
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(widths[0][9846:])

260.77728739097813

In [119]:
np.mean(25.0*(LZM[:149]-LZC[:149]))

309.73154362416108

In [120]:
np.mean(25.0*(LZM[149:]-LZC[149:]))

552.08333333333337

# First segment (Jutai A)

## Estimate friction factor Cf

In [121]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 1293
i2 = 9846
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [122]:
# this might take a while to run
kl = 3.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[0][:9846])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.00760703125


In [39]:
Cf_opt = 0.00760703125

## Estimate migration rate constant kl

In [123]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
for i in np.arange(1,10):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(1,10),errors);

1
2
3
4
5
6
7
8
9


In [54]:
kl_opt = 4.0 # the error is at minimum for kl = 4.0

In [124]:
310/25.0

12.4

In [125]:
plt.figure()
plt.plot(W*kl_opt*curv_t)
plt.plot(migr_rate_t)

## Plot actual migration rate against nominal migration rate

In [126]:
# kernel density and scatterplot of actual vs. nominal migration rate
w = np.nanmean(widths[0][:9846]) 

curv_nodim = W*curv_t*kl_opt
lag = 12
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 2.5
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--') 
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [127]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.860343681877
0.740191250946
0.0


In [128]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

8541

In [129]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [130]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r')

In [131]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.853626448057
0.728678112822
0.0


In [132]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

1.746666174998416

In [70]:
# plot actual vs. predicted migration rate
max_m = 2.5
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [133]:
# plot actual vs. predicted migration rate
max_m = 4.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::5],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::5],c='r',s=15)
for i in cutoff_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::5],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::5],c='g',s=15)

# Second segment (Jutai B)

## Estimate friction factor Cf

In [82]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 9846
i2 = len(x)-1
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [83]:
# this might take a while to run
kl = 4.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[0][9846:])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.00682734375


In [84]:
Cf_opt = 0.00682734375

## Estimate migration rate constant kl

In [85]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
for i in np.arange(1,10):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(1,10),errors);

1
2
3
4
5
6
7
8
9


In [86]:
kl_opt = 4.0 # the error is at minimum for kl = 4.0

In [87]:
552/25.0 # lag

22.08

In [88]:
plt.figure()
plt.plot(W*kl_opt*curv_t)
plt.plot(migr_rate_t)

## Plot actual migration rate against nominal migration rate

In [90]:
# kernel density and scatterplot of actual vs. nominal migration rate
w = np.nanmean(widths[0][9846:]) 

curv_nodim = W*curv_t*kl_opt
lag = 22
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 3.0
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--') 
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [91]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.717197986136
0.514372951318
0.0


In [92]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

8045

In [93]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [94]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r')

In [95]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.691878609838
0.478696010751
0.0


In [96]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

2.2412418121523747

In [98]:
# plot actual vs. predicted migration rate
max_m = 3.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [101]:
# plot actual vs. predicted migration rate
max_m = 5.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::10],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::10],c='r',s=15)
for i in cutoff_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::10],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::10],c='g',s=15)